# Arbolado IA
-------

### Paquetes
---

In [13]:
# General
import ast
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.stats as stats

from time import time
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# preprocessing
from sklearn.preprocessing import LabelEncoder

# metrics
from sklearn.metrics import (accuracy_score, recall_score, precision_score, f1_score)
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import RocCurveDisplay

# models
# from sklearn.linear_model import Perceptron, LogisticRegressionCV
# from sklearn.naive_bayes import GaussianNB
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from sklearn.tree import DecisionTreeClassifier, plot_tree
# from sklearn.cluster import KMeans
# from sklearn.decomposition import PCA

import xgboost as xgb
from xgboost import XGBRegressor 

# model_selection
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedShuffleSplit

### Carga de datos
----

In [51]:
# Carga de datos
df = pd.read_csv('./data-arbolado-junior.csv')

In [52]:
print("El tamaño del DataFrame es: \n")
print(f'Shape: {df.shape}')

df.head()

El tamaño del DataFrame es: 

Shape: (3823, 28)


,id,fecha,latitud,longitud,direccion,barrioNombre,altura,fuste,faseVital,estadoFitosanitario,...,manejoInadecuado,intervenciones,distForestalAdecuada,interf,cazuela,levantamientoVereda,observaciones,especieNombre,lugarRelevamientoNombre,tipoLugarNombre
0,55216,7/2/2018,-31.418802,-64.170274,906,Juniors,Entre 4m. y 7m.,"Entre 0,10m. y 0,40m.",Maduro,Bueno,...,No,Mantenimiento,True,False,Inadecuada,Leve,NaN,Palmera Pindó,Americo Vespucio 900-1000,Calle
1,55217,7/2/2018,-31.418807,-64.170212,916,Juniors,Entre 4m. y 7m.,"Entre 0,10m. y 0,40m.",Maduro,Regular,...,Leve,Mantenimiento,True,True,Adecuada,Leve,NaN,Siempre Verde,Americo Vespucio 900-1000,Calle
2,55218,7/2/2018,-31.418807,-64.170161,924,Juniors,Menor a 4m.,"Entre 0,10m. y 0,40m.",Maduro,Bueno,...,Leve,Mantenimiento,True,False,Franja Verde,No,NaN,Crespón,Americo Vespucio 900-1000,Calle
3,55219,7/2/2018,-31.418832,-64.170114,930,Juniors,Entre 7m. y 10m.,"Entre 0,10m. y 0,40m.",Maduro,Bueno,...,Leve,Mantenimiento,True,False,Franja Verde,No,NaN,Guaran Amarillo,Americo Vespucio 900-1000,Calle
4,55220,7/2/2018,-31.418868,-64.169999,932,Juniors,Menor a 4m.,"Menor a 0,10m.",Maduro,Bueno,...,Leve,Mantenimiento,True,False,Franja Verde,No,NaN,Ficus Benjamina,Americo Vespucio 900-1000,Calle


### Ingeniería de Datos
----

In [53]:
cols_to_drop = [
    "id", "fecha", "latitud", "longitud", "direccion", 
    "barrioNombre", "observaciones", "lugarRelevamientoNombre", 
    "tipoLugarNombre"
]
df = df.drop(columns=cols_to_drop)

In [54]:
print(df.columns)

Index(['altura', 'fuste', 'faseVital', 'estadoFitosanitario', 'inclinacion',
       'ahuecamiento', 'problemaDesrames', 'problemaDescalzado',
       'problemaFustePartido', 'problemaObjetoExtranio', 'problemaGrieta',
       'problemaDesequilibrioCopa', 'manejoInadecuado', 'intervenciones',
       'distForestalAdecuada', 'interf', 'cazuela', 'levantamientoVereda',
       'especieNombre'],
      dtype='object')


In [55]:
label_maps = {}

le = LabelEncoder()

for col in df.columns:
    # Ignora columnas completamente vacías
    if df[col].dropna().empty:
        continue

    first_val = df[col].dropna().iloc[0]

    # Columnas booleanas (True/False)
    if isinstance(first_val, bool):
        df[col] = df[col].astype('boolean').fillna(False).astype(int)

    # Columnas de texto
    elif isinstance(first_val, str):
        df[col] = le.fit_transform(df[col].astype(str))
        # Guardar leyenda: valor original -> número
        label_maps[col] = dict(zip(le.classes_, le.transform(le.classes_)))